# How can I interpolate station data with Kriging in Python?

Reference : https://stackoverflow.com/questions/45175201/how-can-i-interpolate-station-data-with-kriging-in-python

Browsing the web I've found that some tools to use Kriging in Python are pyKriging and Gaussian Process Regression. However, I couldn't make any of them to work.

With OpenTURNS, the KrigingAlgorithm estimates the hyperparameters of a conditioned gaussian process. The purpose of the metamodel that you need is to have the (longitude,latitude) 2D point as input and the precipitation at a given date as output.

The first step is to prepare the data. In the following script, I create the coordinates_train variable which contains the longitude/latitude pairs and the precipitation_train variable which contains the precipitations. I used the precipitations at date 2000/1/2, because the data is missing for 2000/1/1 at station 3.

In [1]:
import openturns as ot
import openturns.experimental as otexp

In [2]:

# Input points
coordinates_train = ot.Sample([[15.68, 62.64], [15.73, 62.12], [16.10, 61.14]])
# Output points
precipitation_train = ot.Sample([[3.4], [2.5], [4.5]])  # At 2000/1/2

Then we can train the kriging. To do this I use a constant basis (the trend of the model) and an exponential covariance model. This should be appropriate, given that the precipitation must be quite regular with respect to the location of the station.

In [3]:
# Fit
inputDimension = 2
basis = ot.ConstantBasisFactory(inputDimension).build()
covarianceModel = ot.SquaredExponential([1.0] * inputDimension, [1.0])
fitter = otexp.GaussianProcessFitter(
    coordinates_train, precipitation_train, covarianceModel, basis
)
fitter.run()
gpr = otexp.GaussianProcessRegression(fitter.getResult())
gpr.run()
gpr_result = gpr.getResult()
krigingMetamodel = gpr_result.getMetaModel()

Then we can use the metamodel to predict the precipitation at a location where it was not recorded. Since krigingMetamodel is a function, I just use the "()" operator.

In [4]:
# Predict
coordinates = [15.70, 62.53]  # A new latitude/longitude pair
precipitation = krigingMetamodel(coordinates)

Then precipitation is a 1D point containing the precipitation at the given location. This is the predicted precipitation.

In [5]:
precipitation

class=Point name=Unnamed dimension=1 values=[3.14956]

You might as well get a more general kriging by having the (longitude,latitude,time) as input. In this case, all you have to do is to add a new dimension to the input training sample, containing the associated times, formatted as real values.